In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-history-questions-dataset/all_data_flattened.json
/kaggle/input/arabic-history-questions-dataset/all_data.json
/kaggle/input/arabic-history-questions-dataset/train.json
/kaggle/input/arabic-history-questions-dataset/test.json
/kaggle/input/arabic-history-questions-dataset/test_raw.json
/kaggle/input/arabic-history-questions-dataset/val_raw.json
/kaggle/input/arabic-history-questions-dataset/train_raw.json
/kaggle/input/arabic-history-questions-dataset/val.json
/kaggle/input/arabart-qa-model/transformers/default/4/__huggingface_repos__.json
/kaggle/input/arabart-qa-model/transformers/default/4/arabart_qa_model/config.json
/kaggle/input/arabart-qa-model/transformers/default/4/arabart_qa_model/training_args.bin
/kaggle/input/arabart-qa-model/transformers/default/4/arabart_qa_model/tokenizer.json
/kaggle/input/arabart-qa-model/transformers/default/4/arabart_qa_model/tokenizer_config.json
/kaggle/input/arabart-qa-model/transformers/default/4/arabart_qa_model/model.safet

In [2]:
#Custom Dataset 
model_path = "/kaggle/input/arabart-wiki-ar-qa-model/transformers/default/3/arabart-wiki-qa/checkpoint-17640"

print("Files in model directory:", os.listdir(model_path))

Files in model directory: ['config.json', 'trainer_state.json', 'training_args.bin', 'tokenizer.json', 'tokenizer_config.json', 'scaler.pt', 'scheduler.pt', 'model.safetensors', 'special_tokens_map.json', 'optimizer.pt', 'rng_state.pth', 'sentencepiece.bpe.model', 'added_tokens.json', 'generation_config.json']


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer with custom special tokens
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=True  # Required for SentencePiece (spiece.model)
)

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

2025-05-16 18:27:41.102797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747420061.578975      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747420061.715931      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
print(tokenizer.special_tokens_map)
# Should show:
# {'additional_special_tokens': ['<context>', '<answer>', '<question>'], ...}

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<sep>']}


In [5]:
!pip install -qU json-repair==0.29.1

In [6]:
import json
from json_repair import json_repair

In [7]:
import json_repair
def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

In [8]:
import json
import json_repair

# Function to parse and repair JSON
def parse_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    try:
        return json_repair.loads(text)  # Repair and parse JSON
    except Exception as e:
        print(f"خطأ في تحليل JSON: {e}")  # Arabic error message
        return None

# # Extract and map each paragraph with a maximum of 3 questions
# def extract_paragraph_question_answer(data, max_questions=3):
#     results = []
#     for item in data:
#         paragraph = item.get("paragraph", "").strip()
#         questions = item.get("questions", [])[:max_questions]  # Limit to 3 questions

#         for q in questions:
#             results.append({
#                 "paragraph": paragraph,
#                 "question": q.get("question", "").strip(),
#                 "answer": q.get("correct_answer", "").strip()
#             })

#     return results

# Load and process the Arabic JSON file
file_path = "/kaggle/input/arabic-history-questions-dataset/test.json"  # استبدل بالمسار الفعلي للملف
data = parse_json(file_path)

# if data:
#     formatted_data = extract_paragraph_question_answer(data, max_questions=3)
#     with open("formatted_output.json", "w", encoding="utf-8") as f:
#         json.dump(formatted_data, f, indent=2, ensure_ascii=False)  # دعم اللغة العربية
#     print("تم تحويل البيانات وحفظها في formatted_output.json")
# else:
#     print("فشل في تحميل بيانات JSON.")


In [9]:
len(data)

682

In [10]:
# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head()

,paragraph,question,answer
0,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,ما هو الهدف الأساسي من إصدار العثمانيين كتيبات...,حث المسلمين على الاتحاد ضد أعداء الإسلام
1,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,من كان الشخص المرشح من قبل البريطانيين ليكون ر...,الشريف حسين
2,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,ماذا كان يتوقع الشريف حسين أن يحقق من خلال تحا...,تأسيس دولة عربية إسلامية تحت نفوذه
3,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,كيف كانت العلاقات بين عبد الله وبين البريطانيي...,كان عبد الله يتبادل الزيارات مع الموظفين البري...
4,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,ما هو الدور الذي كان يتوقع الشريف حسين أن تقوم...,القيام بمساعدة مباشرة ضد العثمانيين


In [11]:
!pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not c

In [12]:
len(df)

682

In [13]:
df.rename(columns={"paragraph": "context"}, inplace=True)
df.head()

,context,question,answer
0,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,ما هو الهدف الأساسي من إصدار العثمانيين كتيبات...,حث المسلمين على الاتحاد ضد أعداء الإسلام
1,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,من كان الشخص المرشح من قبل البريطانيين ليكون ر...,الشريف حسين
2,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,ماذا كان يتوقع الشريف حسين أن يحقق من خلال تحا...,تأسيس دولة عربية إسلامية تحت نفوذه
3,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,كيف كانت العلاقات بين عبد الله وبين البريطانيي...,كان عبد الله يتبادل الزيارات مع الموظفين البري...
4,أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد ...,ما هو الدور الذي كان يتوقع الشريف حسين أن تقوم...,القيام بمساعدة مباشرة ضد العثمانيين


In [14]:
import pandas as pd

# Function to extract only context, question, and actual answer while keeping row count

def extract_relevant_data(dataset):
    return pd.DataFrame({
        "context": dataset["context"],
        "question": dataset["question"],
        "answer": dataset["answer"]  # No need to extract inner "text"
    })
test_data = extract_relevant_data(df)

# Check if the row count remains unchanged

print(len(test_data))  # Should match len(test_data)


682


In [15]:
from datasets import DatasetDict, Dataset, load_dataset
import pandas as pd
def format_for_qa(dataset, dataset_name):
    print(f"\n🔍 Formatting {dataset_name} dataset...")
    formatted_text = dataset["question"] + " <sep> " + dataset["context"]
    
    print("\n📝 Sample Formatted Input (first 200 chars):")
    for i in range(3):
        print(f"Sample {i+1}: {formatted_text.iloc[i][:200]}...")
    
    return pd.DataFrame({
        "text": formatted_text,
        "answer": dataset["answer"]
    })

# Process test data
test_ag = format_for_qa(test_data, "test")  
test_ag_dataset = Dataset.from_pandas(test_ag)

# Create dataset dictionary including train, validation, and test sets
datasets_ag = DatasetDict({
    "test": test_ag_dataset
})



🔍 Formatting test dataset...

📝 Sample Formatted Input (first 200 chars):
Sample 1: ما هو الهدف الأساسي من إصدار العثمانيين كتيبات الدعوة للجهاد؟ <sep> أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد وحث المسلمين على أن يتحدوا ضد أعداء الإسلام وأن يمتنعوا عن تقديم أية مساعدات لهم. وكان...
Sample 2: من كان الشخص المرشح من قبل البريطانيين ليكون رئيسًا صوريا للمسلمين؟ <sep> أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد وحث المسلمين على أن يتحدوا ضد أعداء الإسلام وأن يمتنعوا عن تقديم أية مساعدات لهم...
Sample 3: ماذا كان يتوقع الشريف حسين أن يحقق من خلال تحالفه مع بريطانيا؟ <sep> أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد وحث المسلمين على أن يتحدوا ضد أعداء الإسلام وأن يمتنعوا عن تقديم أية مساعدات لهم. وكا...


In [16]:
datasets_ag

DatasetDict({
    test: Dataset({
        features: ['text', 'answer'],
        num_rows: 682
    })
})

In [17]:
for split in datasets_ag:
    print(f"{split} columns: {datasets_ag[split].column_names}")


test columns: ['text', 'answer']


In [18]:
# datasets_ag['test'] = datasets_ag['test'].remove_columns(['__index_level_0__'])

In [19]:
datasets_ag["test"][0]

{'text': "ما هو الهدف الأساسي من إصدار العثمانيين كتيبات الدعوة للجهاد؟ <sep> أصدر العثمانيون كتيبات بهدف الدعوة إلى الجهاد وحث المسلمين على أن يتحدوا ضد أعداء الإسلام وأن يمتنعوا عن تقديم أية مساعدات لهم. وكان هدف العثمانيين من ذلك أن يتأثر العالم العربى بدعوة الجهاد فينحاز أمير مكة والعرب مختارين إلى صفوفهم ضد الحلفاء. ولقد أيقن الحلفاء عامة والبريطانيون بوجه خاص إزاء دعوة الجهاد وإزاء الدعاية العثمانية بثورة البحث عن رئيس صورى للمسلمين لمقاومة نفوذ السلطان العثمانى وكان الشخص المرشح للقيام بهذا الدور هو الشريف حسين بن على الهاشم، أمير مكة، وأخذ الإنجليز يمنونه بمستقبل باهر ويلوحون له بمنصب الخلافة. وتوقع الحسين، نتيجة لذلك، أن يؤسس دولة عربية إسلامية 'تضم تحت نفوذه الأجزاء العربية من الهلال الخصيب وفى الجزيرة العربية' ولقد حاول الحسين قبل ذلك أن يوقف كل تدخل من جانب حكومة الاتحاديين قد يؤثر على مركز حكومته الذاتية فى مكة، ولكن ذلك أثار غضبهم عليه. ومن ناحية أخرى، شعر الشريف حسين أنه لن يستطيع مقاومة الدولة العثمانية بمفرده، وأنه من الضرورى الحصول على مساعدة دولة كبرى لتحقيق ذلك، واع

In [20]:
def generate_answers(context, questions, num_answers_per_question=1):
    """
    Generates multiple answers for different questions using the same context.
    
    Args:
        context (str): Shared context for all questions
        questions (list): List of different questions to ask about the context
        num_answers_per_question (int): Number of diverse answers to generate per question
    
    Returns:
        dict: {question: [answer1, answer2, ...]}
    """
    results = {}
    
    for question in questions:
        # Format input with both question and context
        input_text = f"{question} <sep> {context}"
        
        # Tokenize with same settings as training
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            max_length=768,
            truncation=True,
            padding="max_length"
        ).to(model.device)
        
        # Generate multiple answers
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.9,
            top_k=50,
            top_p=0.95,
            num_return_sequences=num_answers_per_question
        )
        
        # Decode and clean
        answers = []
        for output in outputs:
            answer = tokenizer.decode(
                output, 
                skip_special_tokens=True
            ).replace(question, "").replace("<sep>", "").strip()
            answers.append(answer)
        
        results[question] = answers
    
    return results

In [21]:
!pip install -U transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [22]:
!pip install rouge-score nltk pandas tqdm
!python -m nltk.downloader punkt
!pip install evaluate rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c847210474faafe38eb8e10b6783d61075698ac2e2eef228605c9b0544ebba93
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
!pip uninstall -y evaluate rouge_score
!rm -rf ~/.cache/huggingface/modules/evaluate_modules


Found existing installation: evaluate 0.4.3
Uninstalling evaluate-0.4.3:
  Successfully uninstalled evaluate-0.4.3
Found existing installation: rouge_score 0.1.2
Uninstalling rouge_score-0.1.2:
  Successfully uninstalled rouge_score-0.1.2


In [24]:
!pip install evaluate==0.4.0 rouge_score


  Using cached rouge_score-0.1.2-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.9 MB/s eta 0:00:00


In [25]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00


In [26]:
print (df.columns)

Index(['context', 'question', 'answer'], dtype='object')


# BERTScore

In [27]:
from sentence_transformers import SentenceTransformer, util
# Load a multilingual embedding model for similarity comparison
embedder = SentenceTransformer('akhooli/Arabic-SBERT-100K')


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/17.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [28]:
from bert_score import score as bert_score

def evaluate_answer_generation_bert(dataset, model, tokenizer, num_samples=10):
    """
    Evaluates answer generation using BERTScore, matching the training format:
    "question <sep> context" → "answer"
    
    Args:
        dataset: Dataset with "text" (question <sep> context) and "answer" fields
        model: Fine-tuned model
        tokenizer: Model tokenizer
        num_samples: Number of samples to evaluate
    
    Returns:
        tuple: (list of results, average BERTScore)
    """
    results = []
    scores = []

    for i, example in enumerate(dataset):
        if i >= num_samples:
            break

        # Extract components from dataset format
        full_text = example["text"]  # "question <sep> context"
        reference_answer = example["answer"].strip()

        # Split into question and context
        try:
            parts = full_text.split("<sep>")
            question = parts[0].strip()
            context = parts[1].strip()
        except IndexError:
            print(f"❌ Skipping sample {i+1} - invalid format")
            continue

        # print(f"\n=== Sample {i+1} ===")
        # print(f"Context: {context[:200]}..." if len(context) > 200 else context)
        # print(f"Question: {question}")
        # print(f"Reference Answer: {reference_answer}")

        # Generate answer
        generated = generate_answers(context, [question], num_answers_per_question=1)
        generated_answer = generated[question][0]
        # print(f"Generated Answer: {generated_answer}")

        # Compute BERTScore
        P, R, F1 = bert_score([generated_answer], [reference_answer], lang="ar")
        score = F1[0].item()
        scores.append(score)

        # print(f"BERTScore (F1): {score:.4f}")

        results.append({
            "sample": i + 1,
            "question": question,
            "context": context,
            "reference_answer": reference_answer,
            "generated_answer": generated_answer,
            "bert_score": score,
        })

    # Final statistics
    if scores:
        avg_score = sum(scores) / len(scores)
        print(f"\n📊 Final Evaluation (Average BERTScore): {avg_score:.4f}")
        print(f"Samples evaluated: {len(scores)}/{num_samples}")
    else:
        avg_score = 0.0
        print("⚠️ No valid samples evaluated")

    return results, avg_score

# Meteor_Score

In [29]:
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')  # Required for METEOR

def evaluate_answer_generation_meteor(dataset, model, tokenizer, num_samples=20):
    results = []
    scores = []
    
    for i, example in enumerate(dataset):
        if i >= num_samples:
            break

        try:
            # Extract and validate input
            full_text = example["text"]
            reference_answer = example["answer"].strip()
            question, context = full_text.split("<sep>")
            question = question.strip()
            context = context.strip()

            # Generate answer
            generated = generate_answers(context, [question], 1)
            generated_answer = generated[question][0]

            # Arabic-aware tokenization with normalization
            def clean_arabic(text):
                text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
                return re.sub(r'\s+', ' ', text).strip()
                
            ref_clean = clean_arabic(reference_answer)
            gen_clean = clean_arabic(generated_answer)
            
            if not ref_clean or not gen_clean:
                raise ValueError("Empty text after cleaning")

            # Calculate METEOR with Arabic-safe parameters
            meteor = meteor_score(
                [ref_clean.split()], 
                gen_clean.split(),
                alpha=0.9,    # Reduce synonym weight
                beta=3,        # Favor recall over precision
                gamma=0        # Disable WordNet
            )
            scores.append(meteor)
            
        except Exception as e:
            print(f"⚠️ Sample {i+1} failed: {str(e)}")
            meteor = 0.0
            
        results.append({
            "sample": i+1,
            "meteor": meteor,
            "ref": reference_answer,
            "gen": generated_answer
        })

    avg_score = np.mean(scores) if scores else 0
    print(f"Final METEOR: {avg_score:.3f} (n={len(scores)})")
    return results, avg_score

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# BLEU

In [30]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import numpy as np

def evaluate_answer_generation_BLEU(dataset, model, tokenizer, num_samples=10):
    """
    Evaluates answer generation using BLEU score with Arabic text support.
    Matches the training format: "question <sep> context" → "answer"
    
    Args:
        dataset: Dataset with "text" (question <sep> context) and "answer" fields
        model: Fine-tuned model
        tokenizer: Model tokenizer
        num_samples: Number of samples to evaluate
    
    Returns:
        tuple: (list of results, average BLEU score)
    """
    # Download required NLTK data
    nltk.download('punkt', quiet=True)
    
    results = []
    scores = []
    smoothing = SmoothingFunction().method2

    for i, example in enumerate(dataset):
        if i >= num_samples:
            break

        # Extract components from dataset format
        full_text = example["text"]  # "question <sep> context"
        reference_answer = example["answer"].strip()

        # Split into question and context
        try:
            question, context = full_text.split("<sep>")
            question = question.strip()
            context = context.strip()
        except ValueError:
            print(f"❌ Skipping sample {i+1} - invalid format")
            continue

        # print(f"\n=== Sample {i+1} ===")
        # print(f"Context: {context[:200]}..." if len(context) > 200 else context)
        # print(f"Question: {question}")
        # print(f"Reference Answer: {reference_answer}")

        # Generate answer
        generated = generate_answers(context, [question], num_answers_per_question=1)
        generated_answer = generated[question][0]
        # print(f"Generated Answer: {generated_answer}")

        # Arabic-aware tokenization
        def arabic_tokenize(text):
            # Simple whitespace tokenizer works better for Arabic than NLTK's default
            return text.split()
        
        ref_tokens = arabic_tokenize(reference_answer)
        gen_tokens = arabic_tokenize(generated_answer)

        # Compute BLEU score with smoothing
        try:
            bleu = sentence_bleu(
                [ref_tokens],
                gen_tokens,
                smoothing_function=smoothing,
                weights=(0.5, 0.5, 0, 0)  # Use 2-gram weighting for Arabic
            )
            scores.append(bleu)
            # print(f"BLEU Score: {bleu:.4f}")
        except Exception as e:
            print(f"⚠️ BLEU calculation failed: {str(e)}")
            bleu = 0.0

        results.append({
            "sample": i + 1,
            "question": question,
            "context": context,
            "reference_answer": reference_answer,
            "generated_answer": generated_answer,
            "bleu_score": bleu,
        })

    # Final statistics
    if scores:
        avg_score = np.mean(scores)
        print(f"\n📊 Final Evaluation (Average BLEU): {avg_score:.4f}")
        print(f"Samples evaluated: {len(scores)}/{num_samples}")
    else:
        avg_score = 0.0
        print("⚠️ No valid scores calculated")

    return results, avg_score

# ROUGE

In [31]:
import re
from rouge_score import rouge_scorer
import numpy as np

def normalize_arabic_text(text):
    """Normalize Arabic text for better evaluation"""
    # Digit normalization
    arabic_to_western = {
        '٠': '0', '١': '1', '٢': '2', '٣': '3', '٤': '4',
        '٥': '5', '٦': '6', '٧': '7', '٨': '8', '٩': '9'
    }
    
    # Common Arabic normalizations
    replacements = [
        ('أ', 'ا'), ('إ', 'ا'), ('آ', 'ا'),  # Alef variations
        ('ة', 'ه'),  # Ta marbuta to ha
        ('ى', 'ي')   # Alif maksura to ya
    ]
    
    # Apply transformations
    for old, new in replacements:
        text = text.replace(old, new)
    text = ''.join([arabic_to_western.get(c, c) for c in text])
    
    # Remove punctuation and extra spaces
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def evaluate_answer_generation_ROUGE(dataset, model, tokenizer, num_samples=10):
    """
    Evaluates answer generation using ROUGE scores with Arabic text support.
    Matches the training format: "question <sep> context" → "answer"
    
    Args:
        dataset: Dataset with "text" (question <sep> context) and "answer" fields
        model: Fine-tuned model
        tokenizer: Model tokenizer
        num_samples: Number of samples to evaluate
    
    Returns:
        tuple: (list of results, average ROUGE-L score)
    """
    # Initialize ROUGE scorer (without stemming for Arabic)
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=False)
    
    results = []
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for i, example in enumerate(dataset):
        if i >= num_samples:
            break

        # Extract components from dataset format
        full_text = example["text"]  # "question <sep> context"
        reference_answer = example["answer"].strip()

        # Split into question and context
        try:
            question, context = full_text.split("<sep>")
            question = question.strip()
            context = context.strip()
        except ValueError:
            print(f"❌ Skipping sample {i+1} - invalid format")
            continue

        # print(f"\n=== Sample {i+1} ===")
        # print(f"Context: {context[:200]}..." if len(context) > 200 else context)
        # print(f"Question: {question}")
        # print(f"Reference Answer: {reference_answer}")

        # Generate answer
        generated = generate_answers(context, [question], num_answers_per_question=1)
        generated_answer = generated[question][0]
        # print(f"Generated Answer: {generated_answer}")

        # Normalize Arabic text
        ref_norm = normalize_arabic_text(reference_answer)
        gen_norm = normalize_arabic_text(generated_answer)

        # Calculate ROUGE scores
        scores = scorer.score(ref_norm, gen_norm)

        # Store scores
        rouge1 = scores['rouge1'].fmeasure
        rouge2 = scores['rouge2'].fmeasure
        rougeL = scores['rougeL'].fmeasure
        
        rouge1_scores.append(rouge1)
        rouge2_scores.append(rouge2)
        rougeL_scores.append(rougeL)

        # print(f"ROUGE-1: {rouge1:.4f} | ROUGE-2: {rouge2:.4f} | ROUGE-L: {rougeL:.4f}")

        results.append({
            "sample": i + 1,
            "question": question,
            "context": context,
            "reference_answer": reference_answer,
            "generated_answer": generated_answer,
            "rouge1": rouge1,
            "rouge2": rouge2,
            "rougeL": rougeL,
            "ref_normalized": ref_norm,
            "gen_normalized": gen_norm
        })

    # Compute average scores
    avg_rouge1 = np.mean(rouge1_scores) if rouge1_scores else 0.0
    avg_rouge2 = np.mean(rouge2_scores) if rouge2_scores else 0.0
    avg_rougeL = np.mean(rougeL_scores) if rougeL_scores else 0.0

    print(f"\n📊 Final Evaluation")
    print(f"ROUGE-1 Average: {avg_rouge1:.4f}")
    print(f"ROUGE-2 Average: {avg_rouge2:.4f}")
    print(f"ROUGE-L Average: {avg_rougeL:.4f}")
    print(f"Samples evaluated: {len(rougeL_scores)}/{num_samples}")

    return results, {"rouge1": avg_rouge1, "rouge2": avg_rouge2, "rougeL": avg_rougeL}

# F1_Score

In [32]:
from sklearn.metrics import f1_score
import numpy as np

def evaluate_answer_generation_f1(dataset, model, tokenizer, num_samples=20):
    """
    Evaluates answer generation using token-level F1 score with Arabic text support.
    Matches the training format: "question <sep> context" → "answer"
    
    Args:
        dataset: Dataset with "text" (question <sep> context) and "answer" fields
        model: Fine-tuned model
        tokenizer: Model tokenizer
        num_samples: Number of samples to evaluate
    
    Returns:
        tuple: (list of results, average F1 score)
    """
    def arabic_tokenize(text):
        """Simple Arabic-aware tokenizer"""
        # Remove punctuation and normalize spaces
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text.split()
    
    results = []
    f1_scores = []

    for i, example in enumerate(dataset):
        if i >= num_samples:
            break

        # Extract components from dataset format
        full_text = example["text"]  # "question <sep> context"
        reference_answer = example["answer"].strip()

        # Split into question and context
        try:
            question, context = full_text.split("<sep>")
            question = question.strip()
            context = context.strip()
        except ValueError:
            print(f"❌ Skipping sample {i+1} - invalid format")
            continue

        # print(f"\n=== Sample {i+1} ===")
        # print(f"Context: {context[:200]}..." if len(context) > 200 else context)
        # print(f"Question: {question}")
        # print(f"Reference Answer: {reference_answer}")

        # Generate answer
        generated = generate_answers(context, [question], num_answers_per_question=1)
        generated_answer = generated[question][0]
        # print(f"Generated Answer: {generated_answer}")

        # Tokenize with Arabic-aware tokenizer
        ref_tokens = set(arabic_tokenize(reference_answer))
        gen_tokens = set(arabic_tokenize(generated_answer))

        # Calculate F1 components
        true_positives = len(ref_tokens & gen_tokens)
        precision = true_positives / len(gen_tokens) if gen_tokens else 0
        recall = true_positives / len(ref_tokens) if ref_tokens else 0
        
        # Compute F1 score
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)

        # print(f"F1 Score: {f1:.4f}")
        # print(f"Precision: {precision:.4f}, Recall: {recall:.4f}")

        results.append({
            "sample": i + 1,
            "question": question,
            "context": context,
            "reference_answer": reference_answer,
            "generated_answer": generated_answer,
            "f1_score": f1,
            "precision": precision,
            "recall": recall,
            "ref_tokens": list(ref_tokens),
            "gen_tokens": list(gen_tokens)
        })

    # Compute statistics
    avg_f1 = np.mean(f1_scores) if f1_scores else 0.0
    print(f"\n📊 Final Evaluation")
    print(f"Average F1: {avg_f1:.4f}")
    print(f"Samples evaluated: {len(f1_scores)}/{num_samples}")

    return results, avg_f1

In [33]:
results, final_score = evaluate_answer_generation_meteor(datasets_ag["test"],
                                                     model,
                                                     tokenizer,
                                                     num_samples=682)

Final METEOR: 0.454 (n=682)


In [34]:
results, final_score = evaluate_answer_generation_bert(datasets_ag["test"],
                                                     model,
                                                     tokenizer,
                                                     num_samples=682)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


📊 Final Evaluation (Average BERTScore): 0.8417
Samples evaluated: 682/682


In [35]:
results, final_score = evaluate_answer_generation_BLEU(datasets_ag["test"],
                                                     model,
                                                     tokenizer,
                                                     num_samples=682)


📊 Final Evaluation (Average BLEU): 0.3806
Samples evaluated: 682/682


In [36]:
results, final_score = evaluate_answer_generation_ROUGE(datasets_ag["test"],
                                                     model,
                                                     tokenizer,
                                                     num_samples=682)


📊 Final Evaluation
ROUGE-1 Average: 0.0420
ROUGE-2 Average: 0.0073
ROUGE-L Average: 0.0420
Samples evaluated: 682/682


In [37]:
results, final_score = evaluate_answer_generation_f1(datasets_ag["test"],
                                                     model,
                                                     tokenizer,
                                                     num_samples=682)


📊 Final Evaluation
Average F1: 0.4518
Samples evaluated: 682/682
